# Writing out a USGSCSM ISD from a PDS3 Mex HRSC image

In [1]:
import ale
from ale.drivers.mex_drivers import MexHrscPds3NaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm
import json
import os
import knoten
import csmapi
import pvl
import pyproj
import numpy as np
import pandas as pd
from pysis import isis

## Instantiating an ALE driver

ALE drivers are objects that define how to acquire common ISD keys from an input image format, in this case we are reading in a PDS3 image using NAIF SPICE kernels for exterior orientation data. If the driver utilizes NAIF SPICE kernels, it is implemented as a [context manager](https://docs.python.org/3/reference/datamodel.html#context-managers) and will furnish metakernels when entering the context (i.e. when entering the `with` block) and free the metakernels on exit. This maintains the integrity of spicelib's internal data structures. These driver objects are short-lived and are input to a formatter function that consumes the API to create a serializable file format. `ale.formatters` contains available formatter functions. 

The default config file is located at `ale/config.yml` and is copied into your home directory at `.ale/config.yml` on first use of the library. The config file can be modified using a text editor. `ale.config` is loaded into memory as a dictionary. It is used to find metakernels for different missions. For example, there is an entry for LRO that points to `/usgs/cpkgs/isis3/data/lro/kernels/mk/` by default. If you want to use your own metakernels, you will need to udpate this path. For example, if the metakernels are located in `/data/lrolrocnac/mk/` the LRO entry should be updated with this path. If you are using the default metakernels, then you do not need to update the path.

ALE has a two step process for writing out an ISD: 1. Instantiate your driver (in this case `LroLrocPds3LabelNaifSpiceDriver`) within a context and 2. pass the driver object into a formatter (in this case, `to_usgscsm`).  

Requirements:
 * A PDS3 LRO LROC NAC image
 * NAIF metakernels installed
 * Config file path for LRO (ale.config.lro) pointing to LRO NAIF metakernel directory 
 * A conda environment with ALE installed into it usisng the `conda install` command or created using the environment.yml file at the base of ALE.

In [2]:
# printing config displays the yaml formatted string
print(ale.config)

# config object is a dictionary so it has the same access patterns 
print('Mex spice directory:', ale.config['mex'])

# updating config for new LRO path in this notebook 
# Note: this will not change the path in `.ale/config.yml`. This change only lives in the notebook.
# ale.config['lro'] = '/data/lrolrocnac/mk/'

cassini: /usgs/cpkgs/isis3/data/cassini/kernels/mk/
dawn: /data/spice/dawn-m_a-spice-6-v1.0/dawnsp_1000/extras/mk
kaguya: /home/arsanders/SELENE/kernels/mk/
lro: /scratch/jlaura/spice/lro-l-spice-6-v1.0/lrosp_1000/extras/mk/
mdis: /data/spice/mess-e_v_h-spice-6-v1.0/messsp_1000/extras/mk
mex: /home/arsanders/testData/5270/
mro: /scratch/spice/mro-m-spice-6-v1.0/mrosp_1000/extras/mk/
spice_root: /data/spice/

Mex spice directory: /home/arsanders/testData/5270/


In [3]:
# change to desired PDS3 image path 
fileName = '/home/arsanders/testData/5270/h5270_0000_ir2.img'

# metakernels are furnsh-ed when entering the context (with block) with a driver instance
# most driver constructors simply accept an image path 
with MexHrscPds3NaifSpiceDriver(fileName) as driver:
    # pass driver instance into formatter function
    print(driver.ephemeris_start_time - driver.ephemeris_stop_time)
    usgscsmString = to_usgscsm(driver)
    lines, times, durations = driver.line_scan_rate
    print(times[-1] - times[0])
    print(driver.fikid)


/home/arsanders/ale/ale/drivers/mex_drivers.py:102: UserWarning: This driver has currently not been tested! Use at your own risk.
  warnings.warn('This driver has currently not been tested! Use at your own risk.')


-203.67427718639374


SpiceyError: 
================================================================================

Toolkit version: N0066

SPICE(EMPTYSTRING) --

String "to" has length zero.

pxform_c

================================================================================

### Write ISD to disk 

ALE formatter functions generally return bytes or a string that can be written out to disk. ALE's USGSCSM formatter function returns a JSON encoded string that can be written out using any JSON library. 

USGSCSM requires the ISD to be colocated with the image file with a `.json` extension in place of the image extension.


In [ ]:
# load the json encoded string ISD
usgscsm_dict = json.loads(usgscsmString)
usgscsm_dict['log_file'] = '/home/arsanders/usgscsm.log'

# strip the image file extension and append .json 
jsonFile = os.path.splitext(fileName)[0] + '.json'

# write to disk 
with open(jsonFile, 'w') as fp:
    json.dump(usgscsm_dict, fp)

In [ ]:
# Set the output location of the resulting .cub
cub_loc = os.path.splitext(fileName)[0] + '.cub'

try: 
    isis.hrsc2isis(from_=fileName, to=cub_loc)
except ProcessError as e:
    print(e.stderr)

try:
    isis.spiceinit(from_=cub_loc, shape='ellipsoid', ckpredicted=True, spkpredicted=True)
except ProcessError as e:
    print(e.stderr)

In [ ]:
camera = knoten.csm.create_csm(fileName)

In [ ]:
def ecef_to_lla(x, y, z, a_radius, b_radius):
    """ Converts from earth-centric, earth-fixed to lat, lon, altitude
    """
    ecef = pyproj.Proj(proj='geocent', a=a_radius, b=b_radius)
    lla = pyproj.Proj(proj='latlong',  a=a_radius, b=b_radius)
    
    lon, lat, alt = pyproj.transform(ecef, lla, x, y, z)
    return lon, lat, alt

In [ ]:
image_pt = csmapi.ImageCoord(.5, .5)
ecef_coord = camera.imageToGround(image_pt, 0.0)
ecef_coord.x, ecef_coord.y, ecef_coord.z
a_rad, b_rad = knoten.csm.get_radii(camera)
lon, lat, _ = ecef_to_lla(ecef_coord.x, ecef_coord.y, ecef_coord.z, a_rad, b_rad)
lon = (lon+360)%360

In [ ]:
pvl_output = pvl.loads(isis.campt(from_=cub_loc, line=1, sample=1))

In [ ]:
# Grab spacecraft position and body fixed look vector from csm
locus = camera.imageToRemoteImagingLocus(csmapi.ImageCoord(1 - .5, 1 - .5))
csm_bodyfixedLV = np.asarray([locus.direction.x, locus.direction.y, locus.direction.z])
csm_position = np.asarray([locus.point.x, locus.point.y, locus.point.z])

# Grab spacecraft position and body fixed look vector from campt pvl output
campt_bodyfixedLV = np.asarray(pvl_output['GroundPoint']['LookDirectionBodyFixed'])
campt_position = pvl_output['GroundPoint']['SpacecraftPosition']
campt_position = np.asarray(campt_position.value) * 1000

# Compute the differences
print(csm_bodyfixedLV - campt_bodyfixedLV)
print(csm_position - campt_position)

In [ ]:
def check_pixel(camera, cub, line, sample):
    """Compares ISIS and USGSCSM pixel.
    
    Takes an image coordinate, projects it to a ground point using ISIS, then projects
    the result back into an image coordinate using USGSCSM and computes the difference
    between image coordinates.
    """
    output = isis.campt(from_=cub, line=line, sample=sample)
    pvl_output = pvl.loads(output)
    bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
    bodyfixed = np.asarray(bodyfixed.value) * 1000
    image_coord = camera.groundToImage(csmapi.EcefCoord(*bodyfixed))
    # (.5,.5) in CSM == (1,1) in ISIS, so we have to subtract (.5,.5) from the ISIS pixels
    line_diff = line - image_coord.line - .5
    sample_diff = sample - image_coord.samp - .5
    return line_diff, sample_diff

In [ ]:
isis_label = pvl.load(cub_loc)
n_samples = isis_label['IsisCube']['Core']['Dimensions']['Samples']
n_lines = isis_label['IsisCube']['Core']['Dimensions']['Lines']

In [ ]:
pixels_dict = {'line' : [1,1,n_lines, n_lines, n_lines/2],
               'sample' : [1, n_samples, 1, n_samples, n_samples/2]}

pixels_df = pd.DataFrame.from_dict(pixels_dict)
pixels_df['line_diff'] = np.NaN
pixels_df['sample_diff'] = np.NaN

for idx, row in pixels_df.iterrows():
    pixels_df.iloc[idx]['line_diff'], pixels_df.iloc[idx]['sample_diff'] = check_pixel(camera, cub_loc, row['line'], row['sample'])

pixels_df

In [ ]:
sample = np.floor(np.linspace(1, n_samples, 100))
result = [check_pixel(camera, cub_loc, 1, i) for i in sample]

In [ ]:
for i in range(len(result)):
    print(result[i+1][1] - result[i][1])